In [200]:
import random
# import pandas as pd
# from IPython.display import display, HTML

import torch

import accelerate
from accelerate import Accelerator

# import huggingface_hub
from transformers import T5ForConditionalGeneration, T5Config, T5TokenizerFast, T5Tokenizer, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

import datasets
from datasets import load_dataset #, load_from_disk
# import evaluate
# from evaluate import load


import tqdm as notebook_tqdm
import os
from dotenv import load_dotenv

In [201]:
accelerator = Accelerator()

In [202]:
load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
# huggingface_token = os.getenv("HF_TOKEN")

# device_type = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device_type)
# print(torch.cuda.device_count())
# # device = torch.device("cpu")

device = accelerator.device

In [203]:
# torch.cuda.empty_cache()
# # torch.cuda.set_per_process_memory_fraction(0.8)

In [204]:
## Load the configuration and set it to output hidden states (OR, can use approach w/: model.config.output_hidden_states = True)
model_checkpoint = "google/flan-t5-small"
# config = T5Config.from_pretrained(model_checkpoint, output_hidden_states=True)

## Load the tokenizer and model with the updated configuration
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint)
# model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
# model = T5ForConditionalGeneration.from_pretrained(model_checkpoint, config=config)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [205]:
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [206]:
raw_datasets = load_dataset("xsum")
# metric = load("rouge")

raw_datasets["train"][0:2]

{'document': ['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreci

In [207]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [208]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b", "google/flan-t5-xl", "google/flan-t5-small"]:
    prefix = "summarize: "
else:
    prefix = ""

In [209]:
batch_size = 500  # Adjust this value based on your available memory
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, padding='longest', truncation=True, max_length=max_input_length, return_tensors="pt")

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], padding='longest', truncation=True, max_length=max_target_length, return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, batch_size=batch_size)
torch.save(tokenized_datasets["train"], 'train_data.pt')
torch.save(tokenized_datasets["validation"], 'valid_data.pt')

In [210]:
train_dataset = torch.load('train_data.pt')
eval_dataset = torch.load('valid_data.pt')
# train_dataset = tokenized_datasets["train"]
# eval_dataset = tokenized_datasets["validation"]

In [257]:
print(train_dataset[0])

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [256]:
preprocess_function(raw_datasets['train'][0:1])['input_ids'].squeeze().shape

torch.Size([541])

In [212]:
# batch_size = 2
# model_name = model_checkpoint.split("/")[-1]
# args = Seq2SeqTrainingArguments(
#     f"{model_name}-finetuned-xsum",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=1,
#     predict_with_generate=True,
#     fp16=True,
#     push_to_hub=False,
# )

In [213]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [259]:
# Set optimizer, training_dataloader, scheduler, dataloader, and loss_function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
loss_function = torc    h.nn.CrossEntropyLoss()


In [260]:
model, optimizer, training_dataloader, scheduler = accelerator.prepare(
    model, optimizer, training_dataloader, scheduler)


In [261]:

# for batch in dataloader:
#     print(batch)

test_batch = next(iter(dataloader))
print((test_batch['input_ids']))

# print the first batch in dataloader
# for batch in dataloader:
#     print(batch["document"])


[tensor([21603]), tensor([10]), tensor([37]), tensor([423]), tensor([583]), tensor([13]), tensor([1783]), tensor([16]), tensor([20126]), tensor([16496]), tensor([6]), tensor([80]), tensor([13]), tensor([8]), tensor([844]), tensor([6025]), tensor([4161]), tensor([6]), tensor([19]), tensor([341]), tensor([271]), tensor([14841]), tensor([5]), tensor([7057]), tensor([161]), tensor([19]), tensor([4912]), tensor([16]), tensor([1626]), tensor([5981]), tensor([11]), tensor([186]), tensor([7540]), tensor([16]), tensor([1276]), tensor([15]), tensor([2296]), tensor([7]), tensor([5718]), tensor([2367]), tensor([14621]), tensor([4161]), tensor([57]), tensor([4125]), tensor([387]), tensor([5]), tensor([15059]), tensor([7]), tensor([30]), tensor([8]), tensor([4653]), tensor([4939]), tensor([711]), tensor([747]), tensor([522]), tensor([17879]), tensor([788]), tensor([12]), tensor([1783]), tensor([44]), tensor([8]), tensor([15763]), tensor([6029]), tensor([1813]), tensor([9]), tensor([7472]), tensor([5

In [262]:
for batch in training_dataloader:
    inputs, targets = batch["input_ids"], batch["labels"]
    inputs = torch.tensor(inputs).squeeze(1).long().to(device)
    targets = torch.tensor(targets).squeeze(1).long().to(device)
    outputs = model(inputs)
    loss = loss_function(outputs, targets.view(-1))  # Convert targets to a tensor with a single dimension
    accelerator.backward(loss)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:


# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     # compute_metrics=compute_metrics
# )

In [ ]:
# args.max_split_size_mb = 10
# trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacty of 14.58 GiB of which 7.56 MiB is free. Process 435192 has 14.57 GiB memory in use. Of the allocated memory 14.38 GiB is allocated by PyTorch, and 51.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF